In [1]:
# !pip install tensorflow

In [2]:
import numpy as np
import mnist

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical

In [3]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

In [4]:
# Нормализация изображений
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

In [5]:
# в вектор длиной 784
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

In [6]:
# создание модели
model = Sequential([
  Dense(64, activation='elu', input_shape=(784,)),  
  Dense(64, activation='elu'),
  Dense(10, activation='sigmoid'),
])

In [7]:
# компиляция модели
model.compile(
  optimizer='nadam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [8]:
# обучение модели
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=25,
  batch_size=30,
)

Epoch 1/25
2000/2000 [==============================] - 11s 5ms/step - loss: 0.3216 - accuracy: 0.9031
Epoch 2/25
2000/2000 [==============================] - 10s 5ms/step - loss: 0.1534 - accuracy: 0.9530
Epoch 3/25
2000/2000 [==============================] - 10s 5ms/step - loss: 0.1156 - accuracy: 0.9641
Epoch 4/25
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0946 - accuracy: 0.9707
Epoch 5/25
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0812 - accuracy: 0.9744
Epoch 6/25
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0723 - accuracy: 0.9771
Epoch 7/25
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0627 - accuracy: 0.9797
Epoch 8/25
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0561 - accuracy: 0.9823
Epoch 9/25
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0520 - accuracy: 0.9834
Epoch 10/25
2000/2000 [==============================] - 10s 5ms/step - l

In [9]:
# оценка модели
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

313/313 [==============================] - 2s 4ms/step - loss: 0.1284 - accuracy: 0.9746


[0.12839962542057037, 0.9746000170707703]

In [10]:
# сохранение модели
model.save_weights('model.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

In [11]:
# прогноз на первых 20 изображениях
predictions = model.predict(test_images[:20])

1/1 [==============================] - 0s 173ms/step


In [12]:
# вывод предсказаний
print(np.argmax(predictions, axis=1))

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [13]:
# правильные ответы
print(test_labels[:20])

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [14]:
predictions = model.predict(test_images[:20])
print(np.argmax(predictions, axis=1))

print(test_labels[:20])

1/1 [==============================] - 0s 40ms/step
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


======================================================================================================================== 

Что я, собственно, делал, помимо (Ctrl+C)+(Ctrl+V):

**1. ОПТИМИЗАТОРЫ**

$А. Adam:$ 

Epoch 5/5
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0844 - accuracy: $0.9732$

313/313 [==============================] - 2s 4ms/step - loss: 0.0897 - accuracy: $0.9718$

$B. RMSprop:$ 
 
Epoch 5/5
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0897 - accuracy: 0.9727

313/313 [==============================] - 2s 4ms/step - loss: 0.1165 - accuracy: 0.9658

$C. Nadam:$

Epoch 5/5
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0808 - accuracy: $0.9741$

313/313 [==============================] - 2s 4ms/step - loss: 0.0964 - accuracy: $0.9714$

$D. SGD$

Epoch 5/5
2000/2000 [==============================] - 9s 5ms/step - loss: 0.2330 - accuracy: 0.9319

313/313 [==============================] - 2s 4ms/step - loss: 0.2185 - accuracy: 0.9343

$E.Ftrl$

Epoch 5/5
2000/2000 [==============================] - 11s 6ms/step - loss: 2.3019 - accuracy: 0.1124

313/313 [==============================] - 2s 4ms/step - loss: 2.3019 - accuracy: 0.1135

**ВЫВОДЫ:**

Хоть и провел анализ оптимизаторов по офф документации $Keras$, я еще недостаточно зрел, чтобы полностью понять их механику.

$Nadam$ и $Adam$ в данном случае победители. Но я бы лично ставил на Adam, потому что у него разница между трейном и тестом минимальна.

$Ftrl$ вообще не туда. Видимо шаг стоит увеличить, раз он не выбрался из локального минимума, но я не могу с точностью утверждать.
Более разреженных данных требует, видимо. Зато он гениально нашел здесь единицу: [7 2 1 0 4] => [1 1 1 1 1].

Ладно, идем дальше.

**2. АКТИВАЦИЯ**

Далее я решил играться с функциями активации. Здесь нейросетка превратилась в еще более черный ящик. Подбираем комбинации опытным путем.

Полностью relu: $0.0980$

Полностью softmax: $0.7106$

Полностью sigmoid: $0.9614$

elu-relu-softmax: $0.9619$

sigmoid-reLU-softmax: $0.9677$

elu-elu-sigmoid: $0.9706$


**ВЫВОДЫ:**

Я только понял, что на выходном слое должна быть softmax или sigmoid, раз мы просчитываем вероятность. Первые два слоя - пальцем в небо.

**3. БАТЧИ**

Размер батча в разумных пределах существенно на метрику не влияет. 

Да, батчей должно быть достаточно для качества обучения.

batch_size=300  ===>  $0.9564$

batch_size=20   ===>  $0.9667$

Ну и не слишком много, чтобы не тратить ресурсы без существенного выигрыша.

batch_size=5   ===> 266s - accuracy: $0.9650$

batch_size=30  ===> 46s - accuracy: $0.9689$

**4. ЭПОХИ**

epochs=5, accuracy: $0.9655$

epochs=7, accuracy: $0.9743$

epochs=10, accuracy: $0.9714$

epochs=20, accuracy: $0.9740$

График точности от эпох легко представить. С увеличением эпох трейн доходит до 0.992 и выше, но на тесте рост замедляется до прямой около 0.975. Оверфиттинг и в случае нейросетей несложно предсказать. Так что 7-10 эпох вполне хватит.

На этом я закончу.